In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import impute
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import median_absolute_error

## Intial Data Opening, replace NAN values.

In [2]:
df = pd.read_csv("../data/data_cleaned.csv")
df.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred_Foot,International_Reputation,...,GKReflexes,Release_Clause,League_Country,League_Level_Pyramid,League_Revenue,League_Revenue_Per_Team,Is_Goalkeeper,Loaned_Out,Off_Work_Rate,Def_Work_Rate
0,31,Argentina,94,94,FC Barcelona,110.5,0.565,2202,Left,5.0,...,8.0,226.5,Spain,1.0,4479.0,223.95,0,0,Medium,Medium
1,33,Portugal,94,94,Juventus,77.0,0.405,2228,Right,5.0,...,11.0,127.1,Italy,1.0,2163.0,108.20,0,0,High,Low
2,26,Brazil,92,93,Paris Saint-Germain,118.5,0.290,2143,Right,5.0,...,11.0,228.1,France,1.0,1692.0,84.60,0,0,High,Medium
3,27,Spain,91,93,Manchester United,72.0,0.260,1471,Right,4.0,...,94.0,138.6,England,1.0,6562.0,328.10,1,0,Medium,Medium
4,27,Belgium,91,92,Manchester City,102.0,0.355,2281,Right,4.0,...,13.0,196.4,England,1.0,6562.0,328.10,0,0,High,High


In [3]:
# replace nan in categorical variables
cat_col = ['Nationality', 'Club', 'Preferred_Foot', 'Off_Work_Rate','Def_Work_Rate', 
           'Real_Face', 'Position', 'Loaned_Out', 'League_Country']
df[cat_col] = df[cat_col].fillna(value='nan')  # fill na for categorical features

In [4]:
# label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for c in cat_col:
    df[c] = le.fit_transform(df[c])

## Pipeline + Modeling Fitting

In [5]:
# Makes Random Forest Object
regressor = RandomForestRegressor(n_estimators=30,
                                  min_samples_leaf=1,
                                  random_state=42)

In [6]:
# Gets lists of numerical columns
numerical_col =  list(set(df.columns) - set(cat_col))
numerical_col.remove('Value')

In [7]:
def make_pipeline(regressor=None):
    '''
    Create a single pipeline that processing the data and then fits the regressor.
    Regressor - Sklearn Model Object we are passing in
    '''
    
    median_impute = impute.SimpleImputer(strategy='median')

    numerical_names = numerical_col
    
    col_transform = compose.ColumnTransformer([('numerics',median_impute,numerical_names)])

    steps_list = [('col_t',col_transform),('clf',regressor)]

    pipeline = Pipeline(steps=steps_list)
    
    return pipeline

In [8]:
# Makes pipeline with random forest
pipeline = make_pipeline(regressor)

## Model Validation

In [14]:
# Makes x and y values
y = df['Value']
X = df.drop('Value',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [15]:
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=make_scorer(median_absolute_error))

In [16]:
np.mean(scores)
scores

array([0.02383333, 0.02566667, 0.02533333, 0.02383333, 0.02416667])

In [17]:
y_pred = pipeline.fit(X_train,y_train).predict(X_test)
median_absolute_error(y_pred,y_test)

0.022416666666666446